In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score,accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, Callback


2023-10-19 23:57:38.883874: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-19 23:57:40.079673: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/nvidia/cudnn/lib:/home/roronoa/.conda/envs/tf/lib/:
2023-10-19 23:57:40.079778: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/

In [2]:
df = pd.read_csv('../Data/CleanedEcommerce.csv')

In [3]:
df.head()

,label,desc
0,Household,paper plane design frame wall hang motiv offic...
1,Household,saf frame paint wood 30 inch x 10 inch special...
2,Household,saf textur modern art print frame paint synthe...
3,Household,saf flower print frame paint synthet 13 5 inch...
4,Household,incred gift india wooden happi birthday uniqu ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27802 entries, 0 to 27801
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   27802 non-null  object
 1   desc    27801 non-null  object
dtypes: object(2)
memory usage: 434.5+ KB


In [10]:
df = df.dropna()

In [11]:
#Text - Input; Label = Output
text = 'desc'
label = 'label'
num_classes = df[label].nunique()

In [12]:
X = df[text].values
y = df[label].values

In [13]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42, stratify=y)





In [15]:
max_words = 1000  # You can adjust this based on your dataset
max_seq_len = 300  # You can adjust this based on your dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


X_train_padded = pad_sequences(X_train_seq, maxlen=max_seq_len)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_seq_len)


In [16]:
#Balancing the Distribution of labels
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_padded, y_train)


In [17]:
X_train_resampled.shape

(33804, 300)

In [23]:
# Define your model
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_seq_len))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

optimizer = Adam(learning_rate=0.001)
early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.03, patience=10, mode='max', restore_best_weights=True)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [24]:
history = model.fit(X_train_resampled, y_train_resampled, epochs=50, batch_size=128, validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50
212/212 [==============================] - 20s 74ms/step - loss: 0.5207 - accuracy: 0.8297 - val_loss: 0.6060 - val_accuracy: 0.8110
Epoch 2/50
212/212 [==============================] - 12s 56ms/step - loss: 0.2846 - accuracy: 0.9287 - val_loss: 0.3008 - val_accuracy: 0.9218
Epoch 3/50
212/212 [==============================] - 12s 55ms/step - loss: 0.2504 - accuracy: 0.9370 - val_loss: 0.3388 - val_accuracy: 0.9117
Epoch 4/50
212/212 [==============================] - 11s 53ms/step - loss: 0.2349 - accuracy: 0.9384 - val_loss: 0.3713 - val_accuracy: 0.9046
Epoch 5/50
212/212 [==============================] - 12s 55ms/step - loss: 0.2107 - accuracy: 0.9470 - val_loss: 0.2873 - val_accuracy: 0.9238
Epoch 6/50
212/212 [==============================] - 12s 56ms/step - loss: 0.1916 - accuracy: 0.9536 - val_loss: 0.2440 - val_accuracy: 0.9284
Epoch 7/50
212/212 [==============================] - 11s 54ms/step - loss: 0.1773 - accuracy: 0.9566 - val_loss: 0.5666 - val_accuracy:

In [25]:
# Predict probabilities for all classes for the test set
y_pred_probabilities = model.predict(X_test_padded)

# Choose the class with the highest probability as the predicted class
y_pred = np.argmax(y_pred_probabilities, axis=1)

# Decode the actual labels to their original class names
y_true_decoded = label_encoder.inverse_transform(y_test)

# Decode the predicted labels to their original class names
y_pred_decoded = label_encoder.inverse_transform(y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_true_decoded, y_pred_decoded)

# Calculate F1 score
f1 = f1_score(y_true_decoded, y_pred_decoded, average='weighted')

# Generate a classification report
report = classification_report(y_true_decoded, y_pred_decoded)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Classification Report:\n", report)

174/174 [==============================] - 3s 12ms/step
Accuracy: 0.9221
F1 Score: 0.9222
Classification Report:
                         precision    recall  f1-score   support

                 Books       0.93      0.92      0.92      1251
Clothing & Accessories       0.95      0.96      0.96      1135
           Electronics       0.87      0.93      0.90      1062
             Household       0.93      0.90      0.92      2113

              accuracy                           0.92      5561
             macro avg       0.92      0.93      0.92      5561
          weighted avg       0.92      0.92      0.92      5561

